In [3]:
# ! pip install --verbose --no-cache-dir torch-scatter
#! pip install --verbose --no-cache-dir torch-sparse
# ! pip install --verbose --no-cache-dir torch-cluster
# ! pip install --verbose --no-cache-dir torch-spline-conv (optional)
# ! pip install torch-geometric

In [4]:
import numpy as np
import torch_geometric as tg
from scipy.sparse import csr_matrix 
import torch
from torch_geometric.data import Data

In [11]:
npzfile = np.load("./preprocessed_data/trade_savez_files.npz", allow_pickle=True)
#_ = npzfile.seek(0)

In [12]:
npzfile.files

['attr_data',
 'attr_shape',
 'sparse_adj_trade',
 'sparse_adj_dists',
 'labels',
 'class_names']

In [13]:
node_attributes = npzfile['attr_data']
attribute_shape = npzfile['attr_shape']
trade_adj = npzfile['sparse_adj_trade']
dist_adj = npzfile['sparse_adj_dists']
class_labels = npzfile['labels']
class_names = npzfile['class_names']

In [14]:
dist_data_adj = dist_adj.tolist()
dist_edge_attr = dist_data_adj.data
dsrc, dtar = dist_data_adj.nonzero()[0], dist_data_adj.nonzero()[1]
dist_edge_index = torch.tensor([dsrc, dtar], dtype = torch.long)

In [15]:
trade_data_adj = trade_adj.tolist()
trade_edge_attr = torch.tensor(trade_data_adj.data)
tsrc, ttar = trade_data_adj.nonzero()[0], trade_data_adj.nonzero()[1]
node_attributes =torch.tensor(node_attributes, dtype = torch.float32)
trade_edge_index = torch.tensor([tsrc, ttar], dtype = torch.long)
y = torch.tensor(class_labels, dtype = torch.long)

In [16]:
node_attributes

tensor([[ 0.0378,  0.0209, -0.1775,  ..., -0.6240,  1.0837,  0.0000],
        [-0.3125, -0.2243, -0.1547,  ..., -0.3690,  0.7236,  0.0000],
        [ 1.0083,  0.0644, -0.1937,  ..., -0.2127,  0.6669,  0.0000],
        ...,
        [-0.0323, -0.0339, -0.1779,  ..., -0.5664,  0.7242,  0.0000],
        [ 0.0938, -0.1178, -0.1912,  ..., -0.4513,  0.7431,  0.0000],
        [-0.1093, -0.1220, -0.1826,  ..., -0.4184,  0.7829,  0.0000]])

In [17]:
trade_data = Data(x = node_attributes, y = y, edge_index = trade_edge_index, edge_attr = trade_edge_attr)
# data.train_idx = torch.tensor([...], dtype=torch.long)
# data.test_mask = torch.tensor([...], dtype=torch.uint8)

In [18]:
# trade_data.num_node_features
# trade_data.test_mask
#trade_edge_index.max().item() + 1
trade_data.x.dtype

torch.float32

In [19]:
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_weight):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        #edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        #print('This is the x for th forward ',x.dtype)
        # Step 3-5: Start propagating messages.
        
        return self.propagate(x = x, edge_index=edge_index, edge_weight=edge_weight, size=(x.size(0), x.size(0)))

    def message(self, x_j, edge_index, edge_weight, size):
        #x_j has shape [E, out_channels]

        #Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
#         print('Deg inv sqrt', deg_inv_sqrt[row].dtype)
#         print('edge weight ', e.dtype)
#         print('deg inv_sqrt col ', deg_inv_sqrt[col].dtype)
        
        norm = deg_inv_sqrt[row] *  edge_weight.float() * deg_inv_sqrt[col]
        #print(x_j.dtype)
        print('Nnorm: ', norm)
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [20]:
import torch.nn.functional as F
#from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(38, 16)
        self.conv2 = GCNConv(16, 4)
        
    def message(self, x_j, norm):
        print('This is where the xj lives')
        return norm.view(-1, 1) * x_j
    
    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = trade_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
# loss.requres_grad = True

for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)#[data.train_mask], data.y[data.train_mask])
    #print(out.shape)
    loss.backward()
    optimizer.step()

Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([       -inf, 193755.8281,  83286.3672,  ...,  11203.4834,
                inf, 725761.3125])
Nnorm:  tensor([   

In [23]:
data.edge_attr

tensor([-2.3114e+04,  1.9083e+07,  8.3652e+06,  ...,  9.9705e+05,
         1.8000e+01,  6.7398e+07], dtype=torch.float64)

In [56]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

torch.float64
torch.float32
Deg inv sqrt torch.float32
edge weight  torch.float64
deg inv_sqrt col  torch.float32


RuntimeError: expected backend CPU and dtype Double but got backend CPU and dtype Float

In [ ]:
data.edge_index.dtype

In [ ]:
trade_edge_attr.dtype

In [ ]:
torch.matmul(data.edge_attr, trade_edge_attr)

In [ ]:
data.y.dtype

In [ ]:
data.y.max()+1

In [ ]:
print(model)

In [ ]:
data.x.dtype

In [ ]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [ ]:
embed_dim = 128
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() + 1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [ ]:
def train():
    model.train()

    loss = 0
    #for data in train_loader:
    data = data.to(device)
    optimizer.zero_grad()
    output = model(data)
    label = data.y.to(device)
    loss = criterion(output, label)
    loss.backward()
    loss = loss.item()
    optimizer.step()
    return loss

device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    train()

In [ ]:
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
labels.append(label)

In [ ]:
for epoch in range(1):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
format(epoch, loss, train_acc, val_acc, test_acc))